In [1]:
import numpy as np
import pandas as pd
import folium
import branca.colormap as cm
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon

In [2]:
trips = pd.read_csv('../data/processed/trips.csv')